In [4]:
import os
from dotenv import load_dotenv
import weaviate
import weaviate.classes.query as wvcq
from weaviate.classes.init import Auth
import json

# --- 1. Connection Details ---
load_dotenv()
WEAVIATE_URL = os.environ.get("WEAVIATE_URL")
WEAVIATE_API_KEY = os.environ.get("WEAVIATE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# --- 2. Connect to Weaviate Cloud (v4 method) ---
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
    headers={"X-OpenAI-Api-Key": OPENAI_API_KEY}
)

# --- 3. Run Queries ---
try:
    products = client.collections.get("EcommerceProducts")

    query_1 = "durable office table"
    print(f'Running Query: "{query_1}"')
    # Query 1: Pure Semantic Search
    response = products.query.near_text(
        query=query_1,
        limit=3,
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_id","product_title", "product_brand", "product_color"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")

    query_2 = "Give me options for different plaques"
    print(f'Running Query: "{query_2}"')
    # Query 1: Pure Semantic Search
    response = products.query.near_text(
        query=query_2,
        limit=5,
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_id","product_title", "product_brand", "product_color"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")
finally:
    client.close()
    print("Connection closed.")

Running Query: "durable office table"
{
  "product_color": "Brown,vintage",
  "product_title": "IRONCK Computer Desk, 55\" Office Desk Simple Study Writing Table, Industrial Style Workstation for Home Office, Easy Assembly, Vintage Brown",
  "product_brand": "IRONCK",
  "product_id": "B07R7QCPMY"
}
  Distance: 0.4148

{
  "product_brand": "Lorell",
  "product_color": "Charcoal",
  "product_title": "Lorell Steel Drawer with Flipper Doors, 48 by 24-Inch, Charcoal",
  "product_id": "B00275YWCS"
}
  Distance: 0.4265

{
  "product_brand": "Generic",
  "product_color": "Brown",
  "product_title": "L-Shaped Desk Table Office Desk Corner Computer Desk 55\u2019\u2019 L Shaped Gaming Work Study Writing Desk for Small Spaces PC Laptop Table for Home Office Bedroom Workstation (Brown)",
  "product_id": "B09DYNLKZH"
}
  Distance: 0.4321

Running Query: "Give me options for different plaques"
{
  "product_brand": "Crown Awards",
  "product_color": "",
  "product_title": "Crown Awards Corporate Plaqu

In [ ]:
# Query 2: Hybrid Search (Semantic + Filter)
print("\n--- 2. Semantic Search for 'dark hoodies' from the brand '11 Degrees' ---")
response = products.query.near_text(
    query="dark hoodies",
    limit=3,
    filters=wvcq.Filter.by_property("product_brand").equal("11 Degrees"),
    return_properties=["product_title", "product_brand"]
)
for item in response.objects:
    print(json.dumps(item.properties, indent=2))


# Query 3: Simple Retrieval with Filter (No Semantic Search)
print("\n--- 3. Find all products with the color 'Blanco' ---")
response = products.query.fetch_objects(
    limit=5,
    filters=wvcq.Filter.by_property("product_color").equal("Blanco"),
    return_properties=["product_title", "product_color"]
)
for item in response.objects:
    print(json.dumps(item.properties, indent=2))